Word Embeddings

1. average the vector for each single word in ImageNet list

In [20]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
import gensim

import pandas as pd
import numpy as np



In [21]:
#As model we use a pretrained model from FastText:
# https://fasttext.cc/docs/en/pretrained-vectors.html
# This model has 6gb of data DISCUSSION: Should we use it?
model = gensim.models.KeyedVectors.load_word2vec_format('./wiki.en.vec', binary=False)

ValueError: not enough values to unpack (expected 2, got 0)

In [16]:
#word2vec_model’ parameter is the loaded word2vec model
#‘words’ parameter is a tokens list (strings) of the sentence / paragraph / document that you want to calculate
def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in word2vec_model.vocab]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return []

In [10]:
df_imgnet = pd.read_csv("../files/ImageNet.csv")
class_list = df_imgnet["Class Name"].tolist()


In [18]:
test = get_mean_vector(model, class_list)

AttributeError: 'Word2Vec' object has no attribute 'vocab'

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api
corpus = api.load('text8')

In [ ]:
for doc in corpus:
    vec = get_mean_vector(model, doc.words)
    if len(vec) > 0:
      # do somthing with the vector ${vec}

2. take zero vectors for words can not be vectorized. for example for hammerhead shark, hammerhead could be zero, but still /2 because two words

First Approach:
Take each item in the ImageNet list, prepare to each dbpedia Class and take the highest score

Second Approach: Average the vector of the items in imageNet class.
In the End we gotTwo sets of results. 
Write a function for top three matchings of dbpedia_class